<a href="https://colab.research.google.com/github/victorviro/Deep_learning_python/blob/master/Regularization_DNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Deep Neural Networks

[Here](https://github.com/victorviro/Deep_learning_python/blob/master/Introduction_artificial_neural_networks_keras.ipynb) we introduced artificial neural networks and trained our first deep neural networks. But they were shallow nets, with just a few hidden layers. What if you need to tackle a complex problem, such as detecting hundreds of types of objects in high-resolution images? You may need to train a much deeper DNN, perhaps with 10 layers or many more, each containing hundreds of neurons, linked by hundreds of thousands of connections. Training a deep DNN isn’t a walk in the park. Here are some of the problems you could run into:

- You may be faced with the tricky *vanishing gradients* problem or the related *exploding gradients* problem. This is when the gradients grow smaller and smaller, or larger and larger, when flowing backward through the DNN during training. Both of these problems make lower layers very hard to train.

- You might not have enough training data for such a large network, or it might be too costly to label.

- Training may be extremely slow.

- A model with millions of parameters would severely risk overfitting the training set, especially if there are not enough training instances or if they are too noisy.

In this notebook we will go through a few popular regularization techniques for large neural networks.

## Avoiding overfitting through regularization

Deep neural networks typically have tens of thousands of parameters, sometimes even millions. With so many parameters, the network has an incredible amount of freedom and can fit a huge variety of complex datasets. But this great flexibility also means that it is prone to overfitting the training set. We need regularization.

Even though [Batch Normalization](https://github.com/victorviro/Deep_learning_python/blob/master/Vanishing_Exploding_gradients_problem_DNNs.ipynb) was designed to solve the vanishing/exploding gradients problems, is also acts like a pretty good regularizer. In this section we will present other popular regularization techniques for neural networks: $l_1$ and $l_2$ regularization, dropout, max-norm regularization and early stopping.



### $l_1$ and $l_2$ regularization

Just like for simple [linear models](https://github.com/victorviro/ML_algorithms_python/blob/master/Introduction_generalized_linear_models.ipynb), you can use $l_1$ and $l_2$ regularization to constrain a neural network’s connection weights (but typically not its biases). Here is how to apply $l_2$ regularization to a Keras layer’s connection weights, using a regularization factor of 0.01:

In [1]:
import keras
layer = keras.layers.Dense(100, activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))

Using TensorFlow backend.


The `l2()` function returns a regularizer that will be called to compute the regularization loss, at each step during training. This regularization loss is then added to the final loss. As you might expect, you can just use `keras.regularizers.l1()` if you want $l_1$ regularization, and if you want both $l_1$ and $l_2$ regularization, use `keras.regularizers.l1_l2()` (specifying both regularization factors).

Let's train a classifier for the [MNIST fashion](https://www.kaggle.com/zalando-research/fashionmnist) problem using $l_2$ regularization.




In [ ]:
# Load and prepare data
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0

from sklearn.model_selection import train_test_split
# Split data data
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.1, shuffle= True)

pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

In [3]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(100, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(10, activation="softmax",
                       kernel_regularizer=keras.regularizers.l2(0.01))
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Train on 54000 samples, validate on 6000 samples
Epoch 1/2
54000/54000 [==============================] - 8s 142us/step - loss: 1.2988 - accuracy: 0.7975 - val_loss: 0.7143 - val_accuracy: 0.8285
Epoch 2/2
54000/54000 [==============================] - 6s 107us/step - loss: 0.7434 - accuracy: 0.8189 - val_loss: 0.7403 - val_accuracy: 0.8207


Since you will typically want to apply the same regularizer to all layers in your network, as well as the same activation function and the same initialization strategy in all hidden layers, you may find yourself repeating the same arguments over and over. This makes it ugly and error-prone. To avoid this, you can try refactoring your code to use loops. Another option is to use Python’s `functools.partial()` function: it lets you create a thin wrapper for any callable, with some default argument values. For example:

In [4]:
from functools import partial

RegularizedDense = partial(keras.layers.Dense,
                           activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Train on 54000 samples, validate on 6000 samples
Epoch 1/2
54000/54000 [==============================] - 6s 110us/step - loss: 1.3192 - accuracy: 0.7975 - val_loss: 0.7765 - val_accuracy: 0.8080
Epoch 2/2
54000/54000 [==============================] - 6s 106us/step - loss: 0.7420 - accuracy: 0.8176 - val_loss: 0.7796 - val_accuracy: 0.7937


### Dropout

[Dropout](https://arxiv.org/abs/1207.0580) is one of the most popular regularization techniques for deep neural networks. It was proposed by Geoffrey Hinton in 2012 and further detailed in a [paper](http://jmlr.org/papers/v15/srivastava14a.html) by Nitish Srivastava, and it has proven to be highly successful: even the state-of-the-art neural networks got a 1–2% accuracy boost simply by adding dropout. This may not sound like a lot, but when a model already has 95% accuracy, getting a 2% accuracy boost means dropping the error rate by almost 40% (going from 5% error to roughly 3%).

It is a fairly simple algorithm: at every training step, every neuron (including the input neurons, but always excluding the output neurons) has a probability $p$ of being temporarily “dropped out,” meaning it will be entirely ignored during this training step, but it may be active during the next step (see Figure 11-9). The hyperparameter $p$ is called the *dropout rate*, and it is typically set to 50%. After training, neurons don’t get dropped anymore. And that’s all (except for a technical detail we will discuss momentarily).

 ![texto alternativo](https://i.ibb.co/TBZSpqV/dropout.png)

It is quite surprising at first that this rather brutal technique works at all. Neurons trained with dropout cannot co-adapt with their neighboring neurons; they have to be as useful as possible on their own. They also cannot rely excessively on just a few input neurons; they must pay attention to each of their input neurons. They end up being less sensitive to slight changes in the inputs. In the end you get a more robust network that generalizes better.

Another way to understand the power of dropout is to realize that a unique neural network is generated at each training step. Since each neuron can be either present or absent, there is a total of $2^N$ possible networks (where $N$ is the total number of droppable neurons). This is such a huge number that it is virtually impossible for the same neural network to be sampled twice. Once you have run a $10000$ training steps, you have essentially trained $10000$ different neural networks (each with just one training instance). These neural networks are obviously not independent since they share many of their weights, but they are nevertheless all different. The resulting neural
network can be seen as an averaging ensemble of all these smaller neural networks.


There is one small but important technical detail. Suppose $p = 50\%$, in which case during testing a neuron will be connected to twice as many input neurons as it was (on average) during training. To compensate for this fact, we need to multiply each neuron’s input connection weights by $0.5$ after training. If we don’t, each neuron will get a total input signal roughly twice as large as what the network was trained on, and it is unlikely to perform well. More generally, we need to multiply each input connection weight by the *keep probability* $(1 – p)$ after training. Alternatively, we can divide each neuron’s output by the keep probability during training (these alternatives are not perfectly equivalent, but they work equally well).


To implement dropout using Keras, you can use the `keras.layers.Dropout` layer.
During training, it randomly drops some inputs (setting them to $0$) and divides the remaining inputs by the keep probability. After training, it does nothing at all, it just passes the inputs to the next layer. For example, the following code applies dropout regularization before every `Dense` layer, using a dropout rate of $0.2$:

In [5]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Train on 54000 samples, validate on 6000 samples
Epoch 1/2
54000/54000 [==============================] - 6s 116us/step - loss: 0.5784 - accuracy: 0.7987 - val_loss: 0.4098 - val_accuracy: 0.8485
Epoch 2/2
54000/54000 [==============================] - 6s 115us/step - loss: 0.4542 - accuracy: 0.8347 - val_loss: 0.3986 - val_accuracy: 0.8503


**Note**: Since dropout is only active during training, the training loss is
penalized compared to the validation loss, so comparing the two can be misleading. In particular, a model may be overfitting the training set and yet have similar training and validation losses. So make sure to evaluate the training loss without dropout (e.g., after training).

If you observe that the model is overfitting, you can increase the dropout rate. Conversely, you should try decreasing the dropout rate if the model underfits the training set. It can also help to increase the dropout rate for large layers, and reduce it for small ones. Moreover, many state-of-the-art architectures only use dropout after the last hidden layer, so you may want to try this if full dropout is too strong.


Dropout does tend to significantly slow down convergence, but it usually results in a much better model when tuned properly. So, it is generally well worth the extra time and effort.

**Note**: If you want to regularize a self-normalizing network based on the
SELU [activation function](https://github.com/victorviro/Deep_learning_python/blob/master/Vanishing_Exploding_gradients_problem_DNNs.ipynb), you should use `AlphaDropout`: this is a variant of dropout that preserves the mean and standard deviation of its inputs (it was introduced in the same paper as SELU, as regular dropout would break self-normalization).

In [6]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])
optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
n_epochs = 20
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Train on 54000 samples, validate on 6000 samples
Epoch 1/20
54000/54000 [==============================] - 5s 97us/step - loss: 0.6697 - accuracy: 0.7566 - val_loss: 0.5752 - val_accuracy: 0.8375
Epoch 2/20
54000/54000 [==============================] - 5s 95us/step - loss: 0.5587 - accuracy: 0.7945 - val_loss: 0.5864 - val_accuracy: 0.8395
Epoch 3/20
54000/54000 [==============================] - 5s 95us/step - loss: 0.5288 - accuracy: 0.8041 - val_loss: 0.5363 - val_accuracy: 0.8420
Epoch 4/20
54000/54000 [==============================] - 5s 96us/step - loss: 0.5084 - accuracy: 0.8115 - val_loss: 0.4493 - val_accuracy: 0.8628
Epoch 5/20
54000/54000 [==============================] - 5s 96us/step - loss: 0.4925 - accuracy: 0.8174 - val_loss: 0.4339 - val_accuracy: 0.8693
Epoch 6/20
54000/54000 [==============================] - 5s 96us/step - loss: 0.4843 - accuracy: 0.8196 - val_loss: 0.4144 - val_accuracy: 0.8688
Epoch 7/20
54000/54000 [==============================] - 5s 95us/ste

In [7]:
model.evaluate(X_test_scaled, y_test)

10000/10000 [==============================] - 1s 52us/step


[0.4830064167499542, 0.8601999878883362]

### Max-Norm regularization

Another regularization technique that is quite popular for neural networks is called *max-norm regularization*: for each neuron, it constrains the weights $\boldsymbol{w}$ of the incoming connections such that $||\boldsymbol{w}||_2 ≤ r$, where $r$ is the max-norm hyperparameter and $||·||_2$ is the $l_2$ norm.
Max-norm regularization does not add a regularization loss term to the overall loss function. Instead, it is typically implemented by computing $||\boldsymbol{w}||_2$ after each training step and clipping $\boldsymbol{w}$ if needed  ($\boldsymbol{w}=\boldsymbol{w}\frac{r}{||\boldsymbol{w}||_2}$).


Reducing $r$ increases the amount of regularization and helps reduce overfitting. Max-norm regularization can also help alleviate the vanishing/exploding gradients problems (if you are not using Batch Normalization). To implement max-norm regularization in Keras, just set every hidden layer’s `kernel_constraint` argument to a `max_norm()` constraint, with the appropriate max value, for example:

In [9]:
MaxNormDense = partial(keras.layers.Dense,
                       activation="selu", kernel_initializer="lecun_normal",
                       kernel_constraint=keras.constraints.max_norm(1.))

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    MaxNormDense(300),
    MaxNormDense(100),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Train on 54000 samples, validate on 6000 samples
Epoch 1/2
54000/54000 [==============================] - 6s 120us/step - loss: 0.5139 - accuracy: 0.8208 - val_loss: 0.3763 - val_accuracy: 0.8612
Epoch 2/2
54000/54000 [==============================] - 6s 116us/step - loss: 0.4022 - accuracy: 0.8538 - val_loss: 0.3982 - val_accuracy: 0.8547


After each training iteration, the model’s `fit()` method will call the object returned by `max_norm()` , passing it the layer’s weights and getting clipped weights in return, which then replace the layer’s weights. You can define your own custom constraint function if you ever need to, and use it as the `kernel_constraint` . You can also constrain the bias terms by setting the `bias_constraint` argument.


The `max_norm()` function has an `axis` argument that defaults to $0$. A `Dense` layer usually has weights of shape [number of inputs, number of neurons], so using `axis=0` means that the max norm constraint will apply independently to each neuron’s weight vector. If you want to use max-norm with convolutional layers, make sure to set the `max_norm()` constraint’s `axis` argument appropriately (usually `axis=[0, 1, 2]`).


### Early stopping

A very different way to regularize iterative learning algorithms such as Gradient Descent is to stop training as soon as the validation error reaches a minimum. This is called *early stopping*.

To implement early stopping simply use the `EarlyStopping` callback. It will interrupt training when it measures no progress on the validation set for
a number of epochs (defined by the `patience` argument). 

In [13]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
early_stopping = keras.callbacks.EarlyStopping(patience=10)

history = model.fit(X_train_scaled, y_train, epochs=2,
                    validation_data=(X_valid, y_valid),
                    callbacks=[early_stopping])

Train on 54000 samples, validate on 6000 samples
Epoch 1/2
54000/54000 [==============================] - 6s 111us/step - loss: 0.5793 - accuracy: 0.8001 - val_loss: 0.7936 - val_accuracy: 0.7053
Epoch 2/2
54000/54000 [==============================] - 6s 108us/step - loss: 0.4515 - accuracy: 0.8360 - val_loss: 0.7053 - val_accuracy: 0.7357
